# Passage Retrieval 실습

이번 과제에서는 3강에서 배운 **Sparse Passage Retrieval**과 4강에서 배운 **Dense Passage Retrieval (DPR)**을 구현해봅니다. Passage Retrieval을 다시 복습해보면,
1. Query와 Passage를 임베딩 시킨 후
2. 임베딩된 벡터들을 각각 dot product해 유사도를 구한 후에
3. 유사도가 가장 높은 Passage들을 검색 대상으로 합니다.   

이 때 임베딩 시키는 방법에서 Sparse와 Dense가 나누어진 점, 다들 기억하시죠?
차근차근 구현해본 후, 전체 Wikipedia에 대해서도 작업해봅시다.

```
🛠 Setup을 하는 부분입니다. 이전 과제에서 반복되는 부분이기 때문에 무지성 실행 하셔도 좋습니다.
💻 실습 코드입니다. 따라가면서 코드를 이해해보세요.
❓ 과제입니다. 주어진 질문과 요구사항에 맞춰서 직접 코드를 짜보세요.
```

## 🛠 초기설정

### 🛠 Requirements

In [15]:
# !pip install tqdm==4.64.1 -q
# !pip install datasets==2.7.0 -q
# !pip install transformers==4.24.0 -q
!pip install scikit-learn==1.1.3 -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### 🛠 난수 고정 및 버전 확인

In [2]:
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)

In [3]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [4]:
print ("PyTorch version:[%s]."% (torch.__version__))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print ("device:[%s]."%(device))

PyTorch version:[2.0.0].
device:[cuda:0].


### 🛠 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [5]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")
corpus = list(set([example["context"] for example in dataset["train"]]))
print(f"총 {len(corpus)}개의 지문이 있습니다.")

Found cached dataset squad_kor_v1 (/home/kingstar/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

총 9606개의 지문이 있습니다.


### 🛠 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, KLUE에서 제공하는 `klue/bert-base` tokenizer를 활용해봅시다. 다른 pretrained 모델을 사용하고 싶으시다면, `model_checkpoint`를 바꿔보세요 !

In [6]:
from transformers import AutoTokenizer

model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

불러온 Tokenzier가 잘 작동하는지 확인해봅시다.

In [9]:
tokenized_input = tokenizer(
    dataset["train"][0]["context"],
    padding="max_length",
    truncation=True
)
pprint(tokenizer.decode(tokenized_input["input_ids"]))

('[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. '
 '이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 '
 '심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 '
 '받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 '
 '라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 '
 '받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 '
 '완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. '
 '결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 '
 '방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 '
 '의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] '
 '[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

## 💻 Ⅰ. Sparse Retriever 실습
첫 번째로 TF-IDF를 통해 임베딩을 만들어봅시다. 이 모듈은 직접 구현할 필요 없이 `sklearn.feature_extract.text`에서 구현된 것을 사용하면, 간단하게 임베딩 시킬 수 있습니다.




### 💻 1. TF-IDF 학습하기
TF-IDF 사용법은 sklearn 홈페이지에서 확인할 수 있습니다. 제공된 링크를 읽어보시면, 아래에 작성된 코드를 쉼게 이해하실 수 있을 거에요.


*   [TF-IDF 공식 홈페이지](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
*   [TF-IDF 사용 예시](https://wikidocs.net/31698)

In [10]:
# Huggingface의 Tokenizer를 사용하셔도 좋고
tokenizer_func = lambda x: tokenizer.tokenize(x)

# 혹은 단순 띄어쓰기 기준으로 Tokenize 하셔도 좋습니다.
# tokenizer_func = lambda x: x.split(' ')

# 어떻게 Tokenize 되었는지 확인해봅시다.
print(f"{'-'*30} 기존 문장 {'-'*30}")
pprint(corpus[20])
print(f"\n{'-'*30} Tokenize 된 문장 {'-'*30}")
pprint(" ".join(tokenizer_func(corpus[20])))

------------------------------ 기존 문장 ------------------------------
('노래는 비연속 4주 동안 정상에 차트 머물렀고, 마지막 4주차 1위 당시 전주보다 157% 상승한 382,000만 건의 디지털 판매량을 '
 '기록하며 가장 높은 주간 판매량을 보였다. 핫 100 에어플레이 차트에서도 1위에 올랐고, 2009년 1월 15일에는 1억 '
 '47,300,000만의 라디오방송량을 기록했다. 핫 알앤비/힙합 송 차트에서도 1위를 했고, 12주 연속 정상을 지켰다. 팝 송 차트, '
 '핫 댄스 클럽 플레이 차트에서도 1위를 했고, 팝 100 차트에서는 2위까지 올라갔다. 〈Single Ladies〉는 400만 건 이상의 '
 '디지털 판매를 하면서 미국 음반 산업 협회 (RIAA)로부터 4× 플래티넘 인증을 받았다. 2012년 10월 닐슨 사운드스캔에 따르면 '
 '미국에서 5,000,000만 건이 넘는 디지털 판매량을 올렸다.')

------------------------------ Tokenize 된 문장 ------------------------------
('노래 ##는 비 ##연 ##속 4 ##주 동안 정상 ##에 차트 머물렀 ##고 , 마지막 4 ##주 ##차 1 ##위 당시 전주 ##보 '
 '##다 157 % 상승 ##한 38 ##2 , 000 ##만 건의 디지털 판매량 ##을 기록 ##하 ##며 가장 높 ##은 주간 판매량 '
 '##을 보였 ##다 . 핫 100 에어 ##플레이 차트 ##에서 ##도 1 ##위 ##에 올랐 ##고 , 2009 ##년 1 ##월 15 '
 '##일 ##에 ##는 1 ##억 47 , 300 , 000 ##만 ##의 라디오 ##방송 ##량 ##을 기록 ##했 ##다 . 핫 알 '
 '##앤 ##비 / 힙합 송 차트 ##에서 ##도 1 ##위 ##를 했 ##고 , 12 ##주 연속 정상 ##을 지켰 ##다 . 팝 송 '
 '차트 , 핫 댄스 클럽 플레이 차트 #

모듈을 활용해서 `fit` 해봅시다. 과정은 어렵지 않습니다.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    tokenizer=tokenizer_func,
    ngram_range=(1,2)
)
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (629 > 512). Running this sequence through the model will result in indexing errors


`Vectorizer`로 임베딩을 시켜주면 (문서의 개수, 단어의 개수) 꼴의 행렬로 변환이 됩니다. 기본적으로 단어의 개수는 지정해주지 않으면 전체 단어의 개수만큼 차원이 지정됩니다. 사용되는 단어의 개수가 너무 많아서 지나치게 행렬이 희소해지면 사용이 불편하기 때문에, 필요하다면 벡터 임베딩 사이즈 또한 지정해줄 수 있습니다.

참고로 결과물이 희소행렬이기 때문에 평소에 사용되는 `numpy.ndarray`가 아닙니다. Scipy 모듈의 csr_matrix를 이용하는데 이는 아래 링크를 참고해주세요
+ [scipy.sparse.csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)
+ [Scipy sparse matrix handling](https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/)

In [12]:
type(sp_matrix)

scipy.sparse._csr.csr_matrix

In [13]:
sp_matrix.shape # (num_passage, num_vocab)

(9606, 684272)

첫 번째 문장의 TF-IDF 벡터를 확인해볼까요?

In [18]:
import pandas as pd

df = pd.DataFrame(
    sp_matrix[0].T.todense(),
    index=vectorizer.get_feature_names_out(),
    columns=["TF-IDF"]
)
df = df.sort_values("TF-IDF", ascending=False)
df.head(10)

,TF-IDF
##함 ##파,0.232869
호 ##함,0.232869
' 호,0.136718
##sp ##m,0.135386
t ##sp,0.135386
호,0.131056
##파 ',0.130868
##sp,0.114001
혹은,0.112909
##파,0.110288


### 💻 2. Query 임베딩하기

이제 Query를 임베딩해봅시다. 아까 사용한 `vectorizer`를 이용하면 됩니다.

In [19]:
sample_idx = random.choice(range(len(dataset["train"])))

query = dataset["train"][sample_idx]["question"]
ground_truth = dataset["train"][sample_idx]["context"]

In [20]:
query_vec = vectorizer.transform([query])

### 💻 3. Dot Product를 통해 유사도 구하기
내적을 통해 주어진 Query와 전체 Passage 사이의 유사도를 구해봅시다.
그리고 값을 내림차순으로 나열하여 높은 점수를 가진 Passage들을 확인해봅시다.

In [21]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [22]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [23]:
k = 5
doc_scores[:k], doc_ids[:k]

(array([0.21208908, 0.07221365, 0.0677442 , 0.05575468, 0.05302485]),
 array([6022, 6711, 9382, 2695, 9174], dtype=int32))

잘 뽑았는지 확인해봅시다.

In [26]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
    print(f"Top-{i + 1} passage with score {doc_scores[i]:.4f}")
    doc_id = doc_ids[i]
    print(corpus[doc_id], "\n")

[Search query]
 구식 군인들의 월급인 쌀에 모래와 돌멩이가 들어가있던 사건을 말미암아 일어난 사태의 이름은? 

[Ground truth passage]
1882년 6월 민영익의 귀국 권고로 일시 귀국했다가 다시 되돌아갔다. 7월 23일 한성부에서 구식 군인들의 월급으로 주는 쌀에 모래와 돌멩이 및 썩은 쌀을 주자 여기에 반발한 구식 군인들에 의해 임오군란이 일어났는데, 그는 임오군란 사태 당시 행동을 삼가고 사태의 추이를 예의주시하고 있었다. 그러다가 그해 10월 13일 박영효, 김옥균 일행이 수신사(修信使) 겸 사죄사(辭罪使)로 하는 사절단(使節團)이 파견되자, 그는 박영효와 김옥균 일행이 도쿄에 방문했을 때 사절단의 통역을 맡아보았다. 수신사는 3개월간 일본의 각 기관을 시찰하고 일본의 여·야당 정치 지도자들과 만나 면담하고 각국 사절과도 폭넓게 접촉하여 의견을 교환했다. 한학을 배워 중국어와 일본어의 기초 실력이 있었던 그는 사절단의 통역을 맡아 활약했으며, 박영효 등이 귀국할 때 일본 유학을 마치고 박영효 일행과 함께 귀국했다. 

Top-1 passage with score 0.2121
1882년 6월 민영익의 귀국 권고로 일시 귀국했다가 다시 되돌아갔다. 7월 23일 한성부에서 구식 군인들의 월급으로 주는 쌀에 모래와 돌멩이 및 썩은 쌀을 주자 여기에 반발한 구식 군인들에 의해 임오군란이 일어났는데, 그는 임오군란 사태 당시 행동을 삼가고 사태의 추이를 예의주시하고 있었다. 그러다가 그해 10월 13일 박영효, 김옥균 일행이 수신사(修信使) 겸 사죄사(辭罪使)로 하는 사절단(使節團)이 파견되자, 그는 박영효와 김옥균 일행이 도쿄에 방문했을 때 사절단의 통역을 맡아보았다. 수신사는 3개월간 일본의 각 기관을 시찰하고 일본의 여·야당 정치 지도자들과 만나 면담하고 각국 사절과도 폭넓게 접촉하여 의견을 교환했다. 한학을 배워 중국어와 일본어의 기초 실력이 있었던 그는 사절단의 통역을 맡아 활약했으며, 박영효 등이 귀국할 때 일본 유학을 마치

Ground Truth와 동일한 Passage가 잘 나왔나요? 제법 성능이 나쁘지 않은걸 확인할 수 있네요.   
### ❓ 제시된 두 Tokenizer를 모두 활용해서 결과를 비교해볼까요?
위에서 제시된 Hugginface Tokenizer와 단순 띄어쓰기로 구분된 Tokenizer는 많은 차이를 불러올까요? 직접 코드로 구현해봅시다.

### ❓ SparseRetrieval 코드를 class로 합쳐봅시다.
현재 주어진 노트북으로 Sparse Retrieval을 잘 달성하셨나요? 하지만 코드가 너무 흩어져있어서 다시 재사용하기 많이 어렵겠네요. 이 코드들을 다른 사람들도 잘 활용할 수 있도록 Class로 모듈화 해봅시다.

❗ _Hint_   
Scratch부터 짜는게 많이 어려우신가요? 그럴 떈 만들어야하는 기능들의 파이프라인을 나열하고 하나씩 함수화하는 것이 좋습니다. 순서대로 생각해보면 -
1. Data를 불러옵니다.
2. TF-IDF를 Fitting한 후 Passage를 Transform합니다.
3. Query를 Fitting한 TF-IDF를 통해 Transform합니다.
4. 내적을 통해 유사도를 구하고, 내림차순을 통해 유사한 Passage를 Retrieval 합니다.

위 4단계를 하나씩 함수로 구현한 후 Class로 합치면 훨씬 쉬울 겁니다! 반드시 이 과정을 따르지 않으셔도 됩니다.

In [19]:
# 화이팅
class TfIdfRetrieval:

    def __init__(self, tokenize_fn, data_path):

        """
        1. 여기서 Data를 불러올까요.
        """

        pass

    def get_sparse_embedding(self):

        """
        2. 여기서 주어진 전체 passage들에 대해 TF-IDF를 .fit 해줍시다.
        """

        pass

    def get_relevant_doc(self, query, k=1):

        """
        여기서
            3. Query를 받아서 TF-IDF에 Transform 시켜줍니다.
            4. 전체 Passage에 대한 유사도를 구한 후 상위 k개의 Passage를 반환합니다.
        """
        pass

잘 구현하셨나요? 여러분들이 만든 클래스를 아래와 같이 활용하는데 성공하셨다면 마무리된 것입니다.


[참고] wikipedia_documents.json, 해당 파일을 코랩에 바로 올리시면, 업로드 하시는데 시간이 매우 소요가 됩니다. 해당 [링크](https://drive.google.com/drive/folders/1dSLJ84QmEOtNwvM7zPwnKBLhc5wXJOb2?usp=sharing)에 가서 파일 위에서 오른쪽 버튼 클릭 후, 드라이브에 바로가기 추가 버튼 누르신 후, 내 드라이브에 옮겨진 파일을 구글마운트후, 바로 사용하시면 소요 되는 시간없이 바로 사용가능합니다 :)

In [20]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/Colab Notebooks/data/wikipedia_documents.json'
except ImportError:
    print('Runnning on local machine, not colab.')
    data_path = './wikipedia_documents.json'

Mounted at /content/drive


In [21]:
tokenize_fn = lambda x: tokenizer.tokenize(x)
retriever = TfIdfRetrieval(tokenize_fn=tokenize_fn, data_path=data_path)

In [ ]:
retriever.get_sparse_embedding()
query = "미국의 대통령은 누구인가?"
doc_score, doc_indices = retriever.get_relevant_doc(query, k=3)

In [ ]:
print(f"[Search Query] {query}")

for i, idx in enumerate(doc_indices):
    print(f"Top-{i + 1}th Passage (Index {idx})")
    pprint(retriever.contexts[idx])

직접 만든 코드에 다음과 같은 추가 사항들을 고려해봅시다.   
#### ➕추가 과제: 다양한 기능 추가하기
+ 현재 코드가 **어떻게 동작하고 있는지** 궁금하지 않으신가요? 가령, 코드가 돌고는 있는데 대체 어떤 메소드를 수행 중인지, 아니면 시간이 얼마나 걸리는지 등 추가적인 정보를 Logger 에 기록하거나 Prompt 에 찍는 방향으로 더 추가해보세요 !
+ 위에서 시간을 출력하는 코드를 짜보셨다면, 이제 fitting 시간이 적지 않음을 확인할 수 있었습니다. 그리고 passage 가 자주 변경되는 것이 아니라면, 같은 기법을 사용해서 **매번 fitting 을 하는 것은 아주 비효율적**입니다. 임베딩된 passage를 따로 **저장해두고 불러와서 사용하면** 더 효율적이지 않을까요? 이 임베딩들을 `.bin` 파일로 저장하고, 만약 `.bin` 파일이 있다면 TfidfVectorizer 가 fitting 하지 않고, 이 `.bin` 파일을 불러오는 방향으로 코드를 추가해봅시다.
+ 예외 케이스가 있지 않을까요? **query가 `str` 1개가 아니라 2개 이상인 `List[str]`로 주어진 경우에는 어떻게 해야할까요?** 이 상황은 멀리 있지 않고 여러분들이 대회에서 inference 할 때도 필요한 코드입니다. 단순히 반복문을 돌릴 수 있지만, 우리는 행렬곱을 이용하고 있기 때문에 만약 query 가 여러 개면 이 행렬곱의 장점을 이용할 수 있을 것 같습니다.
    + 한 개의 query(`str`) 에 대해 유사한 passage 를 구하는 함수와
    + 여러 개의 query(`list`) 에 대해 유사한 passage 를 구하는 함수

  두 가지를 만들어봅시다. 그리고 `retrieve` 라는 함수를 만들어서 query 가 한 개인지 다수인지 체크한 후 각각의 함수를 사용해서 유사한 passage 를 찾도록 수정해봅시다.
+ 또 다른 예외케이스는 `TfidfVectorizer` 에서 생길 수 있습니다. 가령 Passage 에서 한 번도 보지 못한 단어로만 구성된 query 를 입력했다면, TF-IDF 의 특성상 이 단어는 0 으로만 임베딩될 것입니다. 전체 벡터가 0이 되면 어떻게 처리해야할까요? `assert` 를 활용해보세요!
+ 이 메소드는 어떤 parameters를 받아오고 어떤 기능을 하는지 docstring을 추가해봅시다.


요구사항이 너무 많나요? 원래 인생이 그렇습니다.

_Sparse Retrieval 과제는 두 가지 (기존과 추가 과제) 모두 예시 코드가 금요일 (10/15)제공됩니다._

## 💻 Ⅱ. Dense Retriever (BERT) 학습 시키기

이번에는 BERT를 불러오고 학습시키는 과정을 거쳐봅시다. 시작하기 전에 어떤 과정을 거쳐야할 지 생각해봅시다.
1. 데이터셋 준비
2. 모델 준비
3. 학습하기 



### 💻 1. 데이터셋 준비하기
위에서 불러온 `datasets`를 이용해 데이터를 불러왔습니다. 하지만 이번 과제에서 passage encoder가 학습되는 방식은 주어진 query/question과 알맞는 passage를 찾아오는 과정이 필요한데, 이 때 올바른 데이터만 이용해서 학습하는 것이 아니라, 올바르지 않은 passage 또한 학습해보는 과정을 거쳐야합니다. 이를 in-batch negative라고 부르는데 이에 대한 내용은 강의와 아래 논문을 참고해보세요.
+ [Dense Passage Retrieval](https://arxiv.org/abs/2004.04906)

#### 💻 Training Dataset 준비하기 (question, passage pairs)

실습 시간을 단축시키기 위해 일부 데이터만 활용해봅시다.




In [23]:
sample_idx = np.random.choice(range(len(dataset["train"])), 20)
training_dataset = dataset["train"][sample_idx]

#### 💻 Negative sampling을 위한 negative sample들을 샘플링

주어진 query/question에 해당하지 않는 지문들을 뽑아서 훈련데이터로 넣어줍시다. 

In [24]:
# In-batch Negatvie로 사용할 데이터 생성
num_neg = 2

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset["context"]:
    
    while True:
        neg_idxs = np.random.randint(len(corpus), size=num_neg)

        if not c in corpus[neg_idxs]:
            p_neg = corpus[neg_idxs]

            p_with_neg.append(c)
            p_with_neg.extend(p_neg)
            break

주어진 질문에 알맞는 지문과 올바르지 않는 지문을 모두 살펴봅시다.

In [25]:
print("[Query Given]")
pprint(training_dataset["question"][0])

print("\n[Positive context]")
pprint(p_with_neg[0])

print("\n[Negative context]")
pprint(p_with_neg[1])
pprint(p_with_neg[2])

[Query Given]
'문제는 나랏법 자체보다는 법을 적용하고 옹호하는 데 있었다고 묘사한 사학자는?'

[Positive context]
('전원 공동체, 특히 그 중에서도 잉글랜드 동남부는 농노제의 작동과 지역 장원 재판소의 세금 징수를 불만스러워했다. 특히 재판소를 운영하는 '
 '영주들이 곧 원성을 사는 노동 조례나 왕실에서 제정한 법의 현장 집행자로 기능하는 경우가 잦았기 때문에 불만은 가중되었다. 촌락의 엘리트 '
 '계층 다수는 지방정부의 역할을 맡는 것을 거부했고, 재판소의 활동을 방해하기 시작했다. 재판소에 의해 몰수된 가축들은 그 주인들에 의해 '
 '‘해방’되었고, 법관들은 폭행을 당했다. 일부는 전통적인 법을 존중하나 런던에서 내려오는 증오받는 중앙 법률에서는 분리된, 독립적인 촌락 '
 '공동체의 탄생을 지지하기 시작했다. 사학자 미리 루빈은 이 상황을, “문제는 나랏법 그 자체라기보다, 그 법을 적용하고 옹호하는 데 '
 '있었다”고 묘사한다.')

[Negative context]
'2010년 6월 첫 싱글 앨범인 《Push Push》로 데뷔했다. 2011년 5월엔 멤버 보라와 함께 씨스타의 2인조 유닛 그룹 씨스타19을 결성해 싱글 곡 〈Ma Boy〉로 활동하였으며 2011년 10월엔, 효린의 첫 OST인 《영광의 재인》 <내겐 너니까>를 발매하였다. 2012년 1월에는 《드림하이 2》에서 나나 역으로 처음으로 연기를 하였다. 2013년 1월 씨스타19의 첫번째 싱글 앨범인 《있다 없으니까》를 발매했다. 그리고 11월에 드디어 첫 솔로이자, 정규앨범인 《LOVE & HATE》를 발매했다. 2016년 11월 드디어 3년만의 두번째 솔로 앨범이자 첫번째 미니앨범 [EP] 1집인 《IT`S ME》를 발매하였다. 2017년 씨스타 해체 후, 1인 소속사 브리지를 설립하였다. 이후 2018년 2월 6일 씨스타 해체 후 세번째 솔로 앨범으로 컴백을 앞두고 있다.'
'터브먼은 당시 노예 폐지론자들을 상대로 연설을 하면서 한편으로는 자신의 가

처리한 데이터를 `torch`가 처리할 수 있게 `DataLoader`로 넘겨주는 작업을 해봅시다. 기본적으로 Huggingface Pretrained 모델이 `input_ids`, `attention_mask`, `token_type_ids`를 받아주니, 이 3가지를 넣어주도록 합시다.

In [26]:
from torch.utils.data import DataLoader, RandomSampler, TensorDataset

q_seqs = tokenizer(
    training_dataset["question"],
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
p_seqs = tokenizer(
    p_with_neg,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

In [27]:
max_len = p_seqs["input_ids"].size(-1)
p_seqs["input_ids"] = p_seqs["input_ids"].view(-1, num_neg + 1, max_len)
p_seqs["attention_mask"] = p_seqs["attention_mask"].view(-1, num_neg + 1, max_len)
p_seqs["token_type_ids"] = p_seqs["token_type_ids"].view(-1, num_neg + 1, max_len)

print(p_seqs["input_ids"].size())  # (num_example, pos + neg, max_len)

torch.Size([20, 3, 512])


In [28]:
train_dataset = TensorDataset(
    p_seqs["input_ids"], p_seqs["attention_mask"], p_seqs["token_type_ids"], 
    q_seqs["input_ids"], q_seqs["attention_mask"], q_seqs["token_type_ids"]
)

### 💻 2. BERT encoder 학습시키기
1. BERT 모델을 구성한 후
2. Passage를 임베딩하는 `p_encoder`와 Query/Question을 임베딩하는 `q_encoder`를 각각 선언해줍니다.
3. 두 모델을 학습시킵니다.

In [29]:
class BertEncoder(BertPreTrainedModel):

    def __init__(self,
        config
    ):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
      
    def forward(self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [30]:
# Pre-train된 모델을 사용해줍니다. 위에서 사용한 `model_checkpoint`를 재활용합니다.
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

`train` 함수를 정의한 후 `p_encoder`과 `q_encoder`를 학습시켜봅시다.


In [31]:
def train(args, num_neg, dataset, p_encoder, q_encoder):
    batch_size = args.per_device_train_batch_size
  
    # Dataloader
    train_dataloader = DataLoader(dataset, batch_size=batch_size)

    # Optimizer
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {"params": [p for n, p in p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
        {"params": [p for n, p in p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
        {"params": [p for n, p in q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
        {"params": [p for n, p in q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
    ]
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=args.learning_rate,
        eps=args.adam_epsilon
    )
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=args.warmup_steps,
        num_training_steps=t_total
    )

    # Start training!
    global_step = 0

    p_encoder.zero_grad()
    q_encoder.zero_grad()
    torch.cuda.empty_cache()

    train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
    for _ in train_iterator:

        # epoch_iterator = tqdm(train_dataloader, desc="Iteration")
        with tqdm(train_dataloader, unit="batch") as tepoch:
            for batch in tepoch:

                p_encoder.train()
                q_encoder.train()
        
                targets = torch.zeros(batch_size).long() # positive example은 전부 첫 번째에 위치하므로
                targets = targets.to(args.device)

                p_inputs = {
                    "input_ids": batch[0].view(batch_size * (num_neg + 1), -1).to(args.device),
                    "attention_mask": batch[1].view(batch_size * (num_neg + 1), -1).to(args.device),
                    "token_type_ids": batch[2].view(batch_size * (num_neg + 1), -1).to(args.device)
                }
        
                q_inputs = {
                    "input_ids": batch[3].to(args.device),
                    "attention_mask": batch[4].to(args.device),
                    "token_type_ids": batch[5].to(args.device)
                }

                del batch
                torch.cuda.empty_cache()
                # (batch_size * (num_neg + 1), emb_dim)
                p_outputs = p_encoder(**p_inputs)
                # (batch_size, emb_dim)  
                q_outputs = q_encoder(**q_inputs)

                # Calculate similarity score & loss
                p_outputs = p_outputs.view(batch_size, num_neg + 1, -1)
                q_outputs = q_outputs.view(batch_size, 1, -1)

                # (batch_size, num_neg + 1)
                sim_scores = torch.bmm(q_outputs, torch.transpose(p_outputs, 1, 2)).squeeze()  
                sim_scores = sim_scores.view(batch_size, -1)
                sim_scores = F.log_softmax(sim_scores, dim=1)

                loss = F.nll_loss(sim_scores, targets)
                tepoch.set_postfix(loss=f"{str(loss.item())}")

                loss.backward()
                optimizer.step()
                scheduler.step()

                q_encoder.zero_grad()
                p_encoder.zero_grad()

                global_step += 1

                torch.cuda.empty_cache()
                del p_inputs, q_inputs

    return p_encoder, q_encoder

In [32]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2, # 아슬아슬합니다. 작게 쓰세요 !
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01,
)

_혹시 CUDA 메모리 에러가 뜬다면 passage encoder와 query encoder를 나누지 말고 하나로 학습해보세요. `batch_size`도 변경해보세요._

In [33]:
p_encoder, q_encoder = train(
    args,
    num_neg,
    train_dataset,
    p_encoder,
    q_encoder
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


  0%|          | 0/10 [00:00<?, ?batch/s]


  0%|          | 0/10 [00:03<?, ?batch/s, loss=3.0655322074890137]


 10%|█         | 1/10 [00:03<00:34,  3.87s/batch, loss=3.0655322074890137]


 10%|█         | 1/10 [00:04<00:34,  3.87s/batch, loss=1.2814834008167963e-05]


 20%|██        | 2/10 [00:04<00:16,  2.09s/batch, loss=1.2814834008167963e-05]


 20%|██        | 2/10 [00:04<00:16,  2.09s/batch, loss=0.0]                   


 30%|███       | 3/10 [00:05<00:10,  1.52s/batch, loss=0.0]


 30%|███       | 3/10 [00:05<00:10,  1.52s/batch, loss=12.01259708404541]


 40%|████      | 4/10 [00:06<00:07,  1.26s/batch, loss=12.0125970840454

### 💻 3. 검증셋에 있는 Query에 대해 passage-retrieval 해보기
이제 처음 보는 검증셋에 있는 Query와 passage에 대해서도 잘 작동하는지 확인해봅시다. 아래와 같은 순서로 작동해야겠죠?
1. 검증셋을 불러온다.
2. 검증셋의 query와 passage를 임베딩시킨다.
3. 유사도를 통해 유사한 주어진 query에 맞는 passage를 찾는다.

#### 💻 검증셋 불러오기

In [34]:
valid_corpus = list(set([example["context"] for example in dataset["validation"]]))[:10]

sample_idx = random.choice(range(len(dataset["validation"])))
query = dataset["validation"][sample_idx]["question"]
ground_truth = dataset["validation"][sample_idx]["context"]

if not ground_truth in valid_corpus:
    valid_corpus.append(ground_truth)

print(f"{'-'*5} [Selected Query] {'-'*5}")
pprint(query)

print(f"\n{'-'*5} [Ground Truth] {'-'*5}")
pprint(ground_truth)

----- [Selected Query] -----
'SKY가 휴대전화 CM에서 푸시캣 돌스의 노래와 함꼐 춘 춤은?'

----- [Ground Truth] -----
('2005년 9월 13일 푸시캣 돌스의 첫 번째 정규 음반 PCD가 발매됐다. 이 앨범은 그들이 활동했던 댄스팀 스타일의 댄스팝 음악들과, '
 '트리뷰트, 커버곡들로 구성되어 있다. 이 앨범은 뉴질랜드에서 1위를 차지했고, 캐나다, 네덜란드, 미국에서 TOP 5에 이름을 올렸으며, '
 '영국, 오스트리아, 독일, 스위스, 아일랜드에서 10위까지 올라갔다. PCD는 전 세계적으로 900만 장 이상의 판매고를 올렸다. 첫 '
 "싱글 'Don't Cha'는 영국, 오스트레일리아, 캐나다 등의 나라에서 1위에 올랐고, 빌보드 핫 100 차트 2위에 올랐다. 또한 이 "
 "노래는 한국의 휴대전화 기기 제조사 SKY의 휴대전화 CM송으로 쓰여, CM 속 모델들이 추는 춤 이름인 '맷돌춤'의 배경음악으로 "
 '유행하기도 하였다.')


#### 💻 앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성하기

In [35]:
with torch.no_grad():
    p_encoder.eval()
    q_encoder.eval()

    q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors="pt").to("cuda")
    q_emb = q_encoder(**q_seqs_val).to("cpu")  # (num_query, emb_dim)

    p_embs = []
    for p in valid_corpus:
        p = tokenizer(p, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
        p_emb = p_encoder(**p).to("cpu").numpy()
        p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


<ipython-input-35-1e70ac5831e3>:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)


#### 💻 Dot product를 통해 유사도 구하기

In [ ]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

#### 💻 Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [ ]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print(f"Top-{i + 1} passage with score {dot_prod_scores.squeeze()[rank[i]]:.4f}")
  pprint(valid_corpus[rank[i]])

Ground Truth와 똑같거나 비슷한 Passage가 출력됐나요? 역시 Pretrained는 강력하군요.
### ❓ Dense Retrieval 코드를 class로 합쳐봅시다.
현재 구현된 모델은 굉장히 단순히 짜여졌습니다. 이미 Pretrain된 BERT 모델을 활용했는데, 다른 PLM은 어떨까요? 이를 다시 시도해보자니 코드가 너무 셀로 흩어져있어서 재사용이 어렵네요. 우선 클래스화를 진햏한 후에, `BERT`가 아닌 다른 PLM도 활용해봅시다.

❗ _Hint_   
Scratch부터 짜는게 많이 어려우신가요? 그럴 떈 만들어야하는 기능들의 파이프라인을 나열하고 하나씩 함수화하는 것이 좋습니다. 순서대로 생각해보면 -
1. Setup   
    Naive한 `Dataset`을 받아서 이를 In-Batch Negative를 활용한 후 Dataloader로 변경해주는 코드가 있어야겠죠? 클래스 내에서 활용할 수 있도록 속성(attribute)으로 만들어줍시다. 이 코드를 위에서 활용한 `train` 함수에서 조금 차용해볼까요?
2. PLM을 주어진 Passage와 In-batch negative 기법을 활용해서 훈련합니다.   
    이는 위에서 만든 `train` 함수를 약간 응용해서 재활용합시다.
3. 훈련한 PLM을 통해 Query를 Transform합니다.
4. 내적을 통해 유사도를 구하고, 내림차순을 통해 유사한 Passage를 Retrieval 합니다.

위 4단계를 하나씩 함수로 구현한 후 Class로 합치면 훨씬 쉬울 겁니다!

In [ ]:
# 간바레
class DenseRetrieval:

    def __init__(self,
        args,
        dataset,
        num_neg,
        tokenizer,
        p_encoder,
        q_encoder
    ):

        """
        Do
        --
        학습과 추론에 필요한 객체들을 받아서 속성으로 저장합니다.
        객체가 instantiate될 때 in-batch negative가 생긴 데이터를 만들도록 함수를 수행합니다.
        """

        self.args = args
        self.dataset = dataset
        self.num_neg = num_neg

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder.to(args.device)
        self.q_encoder = q_encoder.to(args.device)

        self.prepare_in_batch_negative(num_neg=num_neg)

    def prepare_in_batch_negative(self,
        dataset=None,
        num_neg=2,
        tokenizer=None
    ):

        """
        Args
        ----
        dataset
            Huggingface의 Dataset을 받아오면,
            in-batch negative를 추가해서 Dataloader를 만들어주세요
        """


    def train(self,
        args=None
    ):

        """
        Do
        --
        train을 합니다. 위에 과제에서 이용한 코드를 활용합시다.
        encoder들과 dataloader가 속성으로 저장되어있는 점에 유의해주세요.
        """


    def get_relevant_doc(self,
        query,
        k=1,
        args=None
    ):

        """
        Args
        ----
        query (str)
            문자열로 주어진 질문입니다.

        k (int)
            상위 몇 개의 유사한 passage를 뽑을 것인지 결정합니다.

        args
            Configuration을 필요한 경우 넣어줍니다.
            만약 None이 들어오면 self.args를 쓰도록 짜면 좋을 것 같습니다.

        Do
        --
        1. query를 받아서 embedding을 하고
        2. 전체 passage와의 유사도를 구한 후
        3. 상위 k개의 문서 index를 반환합니다.
        """

In [ ]:
# 데이터셋과 모델은 아래와 같이 불러옵니다.
train_dataset = load_dataset("squad_kor_v1")["train"]

# 메모리가 부족한 경우 일부만 사용하세요 !
num_sample = 1500
sample_idx = np.random.choice(range(len(train_dataset)), num_sample)
train_dataset = train_dataset[sample_idx]

args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)

In [ ]:
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    num_neg=2,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder
)

retriever.train()

query = "유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은?"
results = retriever.get_relevant_doc(query=query, k=5)

In [ ]:
print(f"[Search Query] {query}")

indices = results
for i, idx in enumerate(indices):
    print(f"Top-{i + 1}th Passage (Index {idx})")
    pprint(retriever.dataset['context'][idx])

#### ➕추가 과제: 다양한 기능 추가하기
현재 코드에서 불편한 부분이 있죠. 일단 너무 많은 기능을 한 class에 넣은 것도 문제, 메소드도 한 번에 너무 다양한 기능을 하는 것이 문제입니다. 메소드 내의 기능을 쪼개서 여러 메소드로 나누거나 데이터셋을 만드는 부분 같이 큰 기능을 다른 class로 분리해봅시다. (DPR 추가 과제는 예시 코드가 제공되지 않습니다. 다양하게 활용해보세요 !)

### ❓ Sparse Retrieval과 Dense Retrieval을 둘 다 시도해보았습니다. 어떤 차이가 있을까요? 팀원들과 논의해보세요.

## ❓ [선택] Wikipedia documents에 대해 Passage Retrieval 실습하기

위에서 배운 Passage Retrieval을 Wikipedia에 있는 문서들로 진행해봅시다. 사실 위에서 두 클래스를 성공적으로 만들었다면, 데이터를 불러온 후 적용만 시키면 끝이 납니다.   
***별도의 예시코드가 제공되지 않습니다**

In [ ]:
# 츄라이 츄라이

만족스러운 결과가 나왔나요? 그렇지 않았다면 모델의 구조를 바꾸거나, 더 많은 training set으로 학습 시켜보세요 !

### ❓ Take-home Question
이 QA 모델을 배포한다고 생각해봅시다. 성능이 좋아서 점점 이용자가 많아지는 상황입니다. 만약 동시에 1000명의 이용자가 Query를 보내면 어떻게 될까요? 전체 Passage가 100,000개라면, 이 유사도를 계산하고 비교하는 횟수는 ... 점점 많아질 것 같네요. 이 문제는 어떻게 해결하면 좋을까요?

### **콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다. 모델 라이선스 : MIT License

